In [1]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
import numpy as np
import tqdm
import pickle

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
flow_size=300
# is_training=input('train?')
# TRAINING= True if is_training=='y' else False
TRAINING = True

In [3]:
all_runs={'8872':'192.168.122.117'
          ,'8802':'192.168.122.117','8873':'192.168.122.67','8803':'192.168.122.67',
         '8874':'192.168.122.113','8804':'192.168.122.113','8875':'192.168.122.120',
        '8876':'192.168.122.30','8877':'192.168.122.208','8878':'192.168.122.58'
        }

In [4]:
dataset=[]

for name in all_runs:
    f = open('../dataset/%s_tordata300.pickle'%name,'rb')
    dataset+=pickle.load(f)

In [5]:
len(dataset)

7324

In [8]:
if TRAINING:
    
    
    len_tr=len(dataset)
    train_ratio=float(len_tr-6000)/float(len_tr)
    rr= list(range(len(dataset)))
    np.random.shuffle(rr)

    train_index=rr[:int(len_tr*train_ratio)]
    test_index= rr[int(len_tr*train_ratio):] #range(len(dataset_test)) # #
    pickle.dump(test_index,open('test_index300.pickle','wb'))
else:
    test_index=pickle.load(open('test_index300.pickle'))[:1000]

In [9]:
negetive_samples=199

In [10]:
def generate_data(dataset,train_index,test_index,flow_size):
    


    global negetive_samples



    all_samples=len(train_index) #训练集样本个数
    labels=np.zeros((all_samples*(negetive_samples+1),1)) # 200个标签数量
    l2s=np.zeros((all_samples*(negetive_samples+1),8,flow_size,1)) #？？？？

    index=0
    random_ordering=[]+train_index #拷贝
    for i in tqdm.tqdm( train_index):
        #[]#list(lsh.find_k_nearest_neighbors((Y_train[i]/ np.linalg.norm(Y_train[i])).astype(np.float64),(50)))

        l2s[index,0,:,0]=np.array(dataset[i]['here'][0]['<-'][:flow_size])*1000.0
        l2s[index,1,:,0]=np.array(dataset[i]['there'][0]['->'][:flow_size])*1000.0
        l2s[index,2,:,0]=np.array(dataset[i]['there'][0]['<-'][:flow_size])*1000.0
        l2s[index,3,:,0]=np.array(dataset[i]['here'][0]['->'][:flow_size])*1000.0

        l2s[index,4,:,0]=np.array(dataset[i]['here'][1]['<-'][:flow_size])/1000.0
        l2s[index,5,:,0]=np.array(dataset[i]['there'][1]['->'][:flow_size])/1000.0
        l2s[index,6,:,0]=np.array(dataset[i]['there'][1]['<-'][:flow_size])/1000.0
        l2s[index,7,:,0]=np.array(dataset[i]['here'][1]['->'][:flow_size])/1000.0


        # if index % (negetive_samples+1) !=0:
        #     print(index , len(nears))
        #     raise
        labels[index,0]=1 #相关的标记为1
        m=0
        index+=1
        # 接下来对负样本进行处理
        np.random.shuffle(random_ordering)
        for idx in random_ordering:
            if idx==i or m>(negetive_samples-1):
                continue

            m+=1

            l2s[index,0,:,0]=np.array(dataset[idx]['here'][0]['<-'][:flow_size])*1000.0
            l2s[index,1,:,0]=np.array(dataset[i]['there'][0]['->'][:flow_size])*1000.0
            l2s[index,2,:,0]=np.array(dataset[i]['there'][0]['<-'][:flow_size])*1000.0
            l2s[index,3,:,0]=np.array(dataset[idx]['here'][0]['->'][:flow_size])*1000.0

            l2s[index,4,:,0]=np.array(dataset[idx]['here'][1]['<-'][:flow_size])/1000.0
            l2s[index,5,:,0]=np.array(dataset[i]['there'][1]['->'][:flow_size])/1000.0
            l2s[index,6,:,0]=np.array(dataset[i]['there'][1]['<-'][:flow_size])/1000.0
            l2s[index,7,:,0]=np.array(dataset[idx]['here'][1]['->'][:flow_size])/1000.0

            #l2s[index,0,:,0]=Y_train[i]#np.concatenate((Y_train[i],X_train[idx]))#(Y_train[i]*X_train[idx])/(np.linalg.norm(Y_train[i])*np.linalg.norm(X_train[idx]))
            #l2s[index,1,:,0]=X_train[idx]



            labels[index,0]=0
            index+=1




    #lsh.setup((X_test / np.linalg.norm(X_test,axis=1,keepdims=True)) .astype(np.float64))
    index_hard=0
    num_hard_test=0
    l2s_test=np.zeros((len(test_index)*(negetive_samples+1),8,flow_size,1))
    labels_test=np.zeros((len(test_index)*(negetive_samples+1)))
    l2s_test_hard=np.zeros((num_hard_test*num_hard_test,2,flow_size,1))
    index=0
    random_test=[]+test_index

    for i in tqdm.tqdm(test_index):
        #list(lsh.find_k_nearest_neighbors((Y_test[i]/ np.linalg.norm(Y_test[i])).astype(np.float64),(50)))



        # if index % (negetive_samples+1) !=0:
        #     print(index, nears)
        #     raise 
        m=0

        np.random.shuffle(random_test)
        for idx in random_test:
            if idx==i or m>(negetive_samples-1):
                continue

            m+=1
            l2s_test[index,0,:,0]=np.array(dataset[idx]['here'][0]['<-'][:flow_size])*1000.0
            l2s_test[index,1,:,0]=np.array(dataset[i]['there'][0]['->'][:flow_size])*1000.0
            l2s_test[index,2,:,0]=np.array(dataset[i]['there'][0]['<-'][:flow_size])*1000.0
            l2s_test[index,3,:,0]=np.array(dataset[idx]['here'][0]['->'][:flow_size])*1000.0

            l2s_test[index,4,:,0]=np.array(dataset[idx]['here'][1]['<-'][:flow_size])/1000.0
            l2s_test[index,5,:,0]=np.array(dataset[i]['there'][1]['->'][:flow_size])/1000.0
            l2s_test[index,6,:,0]=np.array(dataset[i]['there'][1]['<-'][:flow_size])/1000.0
            l2s_test[index,7,:,0]=np.array(dataset[idx]['here'][1]['->'][:flow_size])/1000.0
            labels_test[index]=0
            index+=1

        l2s_test[index,0,:,0]=np.array(dataset[i]['here'][0]['<-'][:flow_size])*1000.0
        l2s_test[index,1,:,0]=np.array(dataset[i]['there'][0]['->'][:flow_size])*1000.0
        l2s_test[index,2,:,0]=np.array(dataset[i]['there'][0]['<-'][:flow_size])*1000.0
        l2s_test[index,3,:,0]=np.array(dataset[i]['here'][0]['->'][:flow_size])*1000.0

        l2s_test[index,4,:,0]=np.array(dataset[i]['here'][1]['<-'][:flow_size])/1000.0
        l2s_test[index,5,:,0]=np.array(dataset[i]['there'][1]['->'][:flow_size])/1000.0
        l2s_test[index,6,:,0]=np.array(dataset[i]['there'][1]['<-'][:flow_size])/1000.0
        l2s_test[index,7,:,0]=np.array(dataset[i]['here'][1]['->'][:flow_size])/1000.0
        #l2s_test[index,2,:,0]=dataset[i]['there'][0]['->'][:flow_size]
        #l2s_test[index,3,:,0]=dataset[i]['here'][0]['<-'][:flow_size]

        #l2s_test[index,0,:,1]=dataset[i]['here'][1]['->'][:flow_size]
        #l2s_test[index,1,:,1]=dataset[i]['there'][1]['<-'][:flow_size]
        #l2s_test[index,2,:,1]=dataset[i]['there'][1]['->'][:flow_size]
        #l2s_test[index,3,:,1]=dataset[i]['here'][1]['<-'][:flow_size]
        labels_test[index]=1

        index+=1
    return l2s, labels,l2s_test,labels_test

In [11]:
def model(flow_before,dropout_keep_prob):
    last_layer=flow_before
    flat_layers_after=[flow_size*2,1000,50,1] #[600,1000,50,1]
    for l in range(len(flat_layers_after)-1):
        flat_weight = tf.get_variable("flat_after_weight%d"%l, [flat_layers_after[l],flat_layers_after[l+1]],
        initializer=tf.random_normal_initializer(stddev=0.01,mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d"%l, [flat_layers_after[l+1]],
        initializer=tf.zeros_initializer())

        _x=tf.add(
                tf.matmul(last_layer, flat_weight),
                flat_bias)
        if l<len(flat_layers_after)-2:
            _x=tf.nn.dropout(tf.nn.relu(_x,name='relu_noise_flat_%d'%l),keep_prob=dropout_keep_prob)
        last_layer=_x
    return last_layer

In [12]:
def model_cnn(flow_before,dropout_keep_prob):
    last_layer=flow_before
    
    CNN_LAYERS=[[2,20,1,2000,5],[4,10,2000,800,3]]
    
    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable("cnn_weight%d"%cnn_size, CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d"%cnn_size, [CNN_LAYERS[cnn_size][-2]],
            initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer, cnn_weights, strides=[1, 2,2, 1], padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x,name='relu_cnn_%d'%cnn_size)
        pool = tf.nn.max_pool(conv, ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1], strides=[1, 1, 1, 1],padding='VALID')
        last_layer=pool
    last_layer=tf.reshape(last_layer, [batch_size,-1])
    
    flat_layers_after=[49600,3000,800,100,1]
    for l in range(len(flat_layers_after)-1):
        flat_weight = tf.get_variable("flat_after_weight%d"%l, [flat_layers_after[l],flat_layers_after[l+1]],
        initializer=tf.random_normal_initializer(stddev=0.01,mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d"%l, [flat_layers_after[l+1]],
        initializer=tf.zeros_initializer())

        _x=tf.add(
                tf.matmul(last_layer, flat_weight),
                flat_bias)
        if l<len(flat_layers_after)-2:
            _x=tf.nn.dropout(tf.nn.relu(_x,name='relu_noise_flat_%d'%l),keep_prob=dropout_keep_prob)
        last_layer=_x
    return last_layer

In [14]:
if TRAINING:
    batch_size=64
    learn_rate=0.0001

    graph = tf.Graph()
    with graph.as_default():#8,flow_size,1表示height，width，inchannel
        train_flow_before = tf.placeholder(tf.float32, shape=[batch_size, 8,flow_size,1],name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,name='label_placeholder',shape=[batch_size,1])
        dropout_keep_prob = tf.placeholder(tf.float32,name='dropout_placeholder')
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict=tf.nn.sigmoid(y2)
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y2,labels=train_label),name='loss_sigmoid')
        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)
        s_loss=tf.summary.scalar('loss', loss)
        summary_op = tf.summary.merge_all()
        # Add variable initializer.
        init = tf.global_variables_initializer()
        saver = tf.train.Saver()


else:
    batch_size=2804/2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32, shape=[batch_size, 8,flow_size,1],name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,name='label_placeholder',shape=[batch_size,1])
        dropout_keep_prob = tf.placeholder(tf.float32,name='dropout_placeholder')
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict=tf.nn.sigmoid(y2)
        saver = tf.train.Saver()

In [15]:
num_epochs=5
import datetime

writer = tf.summary.FileWriter('./logs', graph=graph)

In [16]:
import os
if TRAINING:
    with tf.Session(graph=graph) as session:
        # We must initialize all variables before we use them.
        session.run(init)
        

        for epoch in range(num_epochs ):
            l2s,labels,l2s_test,labels_test=generate_data(dataset=dataset,train_index=train_index,test_index=test_index,flow_size=flow_size)
            rr= list(range(len(l2s)))
            np.random.shuffle(rr)
            l2s=l2s[rr]
            labels=labels[rr]


            average_loss = 0
            new_epoch=True
            num_steps= (len(l2s)//batch_size)-1

            for step in range(num_steps):
                start_ind = step*batch_size
                end_ind = ((step + 1) *batch_size)
                if end_ind < start_ind:
                    print('HOOY')
                    continue

                else:
                    batch_flow_before=l2s[start_ind:end_ind,:]
                    batch_label= labels[start_ind:end_ind]


                feed_dict = {train_flow_before: batch_flow_before,
                                train_label:batch_label,
                             dropout_keep_prob:0.6}
                # We perform one update step by evaluating the optimizer op (including it
                # in the list of returned values for session.run()

                _, loss_val,summary = session.run([optimizer, loss, summary_op], feed_dict=feed_dict)



                # average_loss += loss_val
                writer.add_summary(summary, (epoch*num_steps) +step)

                # print step, loss_val
                # if step % FLAGS.print_every_n_steps == 0:
                #     if step > 0:
                #         average_loss /= FLAGS.print_every_n_steps
                #     # The average loss is an estimate of the loss over the last 2000 batches.
                #     print("Average loss at step ", step, ": ", average_loss)
                #     average_loss = 0.

                # Note that this is expensive (~20% slowdown if computed every 500 steps)

                if ((epoch*num_steps) +step) % 100 == 0:
                    print("Average loss on validation set at step ",  (epoch*num_steps) +step, ": ", loss_val)
                if (((epoch*num_steps) +step)) % 3000 == 0 and epoch >1:
                    tp=0
                    fp=0

                    num_steps_test= (len(l2s_test)//batch_size)-1
                    Y_est=np.zeros((batch_size*(num_steps_test+1)))
                    for step in range(num_steps_test):
                        start_ind = step*batch_size
                        end_ind = ((step + 1) *batch_size)
                        test_batch_flow_before=l2s_test[start_ind:end_ind]
                        feed_dict = {
                                train_flow_before:test_batch_flow_before,
                            dropout_keep_prob:1.0}


                        est=session.run(predict, feed_dict=feed_dict)
                        #est=np.array([xxx.sum() for xxx in test_batch_flow_before])
                        Y_est[start_ind:end_ind]=est.reshape((batch_size))
                    num_samples_test=int(len(l2s_test)/(negetive_samples+1))

                    for idx in range(num_samples_test-1):
                        best=np.argmax(Y_est[idx*(negetive_samples+1):(idx+1)*(negetive_samples+1)])

                        if labels_test[best+(idx*(negetive_samples+1))]==1:
                            tp+=1
                        else:
                            fp+=1
                    print (tp,fp)
                    acc= float(tp)/float(tp+fp)
                    if float(tp)/float(tp+fp)>0.8:      
                        print ('saving...')
                        save_path = saver.save(session, "./model/tor_199_epoch%d_step%d_acc%.2f.ckpt"%(epoch,step,acc))
                        print ('saved')
            print ('Epoch',epoch)
            #save_path = saver.save(session, "/mnt/nfs/scratch1/milad/model_diff_large_1e4_epoch%d.ckpt"%(epoch))

            #t.join()
else:
    with tf.Session(graph=graph) as session:
        name=input('model name')
        saver.restore(session, "/mnt/nfs/work1/amir/milad/%s"%name)
        print("Model restored.")
        corrs=np.zeros((len(test_index),len(test_index)))
        batch=[]
        l2s_test_all=np.zeros((batch_size,8,flow_size,1))
        l_ids=[]
        index=0
        xi,xj=0,0
        for i in tqdm.tqdm(test_index):
            xj=0
            for j in test_index:
                
                l2s_test_all[index,0,:,0]=np.array(dataset[j]['here'][0]['<-'][:flow_size])*1000.0
                l2s_test_all[index,1,:,0]=np.array(dataset[i]['there'][0]['->'][:flow_size])*1000.0
                l2s_test_all[index,2,:,0]=np.array(dataset[i]['there'][0]['<-'][:flow_size])*1000.0
                l2s_test_all[index,3,:,0]=np.array(dataset[j]['here'][0]['->'][:flow_size])*1000.0

                l2s_test_all[index,4,:,0]=np.array(dataset[j]['here'][1]['<-'][:flow_size])/1000.0
                l2s_test_all[index,5,:,0]=np.array(dataset[i]['there'][1]['->'][:flow_size])/1000.0
                l2s_test_all[index,6,:,0]=np.array(dataset[i]['there'][1]['<-'][:flow_size])/1000.0
                l2s_test_all[index,7,:,0]=np.array(dataset[j]['here'][1]['->'][:flow_size])/1000.0
                l_ids.append((xi,xj))
                index+=1
                if index==batch_size:
                    index=0
                    cor_vals=session.run(predict,feed_dict={train_flow_before:l2s_test_all,
                            dropout_keep_prob:1.0})
                    for ids in range(len(l_ids)):
                        di,dj=l_ids[ids]
                        corrs[di,dj]=cor_vals[ids]
                    l_ids=[]
                xj+=1
            xi+=1
        np.save(open('correlation_values_test.np','w'),corrs)

2024-03-11 10:53:14.785293: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-11 10:53:14.785700: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-11 10:53:14.786031: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-11 10:53:14.786357: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-11 10:53:14.824655: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

Average loss on validation set at step  0 :  0.7592377
Average loss on validation set at step  100 :  0.064491
Average loss on validation set at step  200 :  0.005175244
Average loss on validation set at step  300 :  0.0026024308
Average loss on validation set at step  400 :  0.00049904076
Average loss on validation set at step  500 :  0.021806583
Average loss on validation set at step  600 :  0.012871393
Average loss on validation set at step  700 :  0.13550834
Average loss on validation set at step  800 :  0.18328866
Average loss on validation set at step  900 :  0.0049697654
Average loss on validation set at step  1000 :  0.0056510633
Average loss on validation set at step  1100 :  0.008553928
Average loss on validation set at step  1200 :  0.013115223
Average loss on validation set at step  1300 :  0.008238776
Average loss on validation set at step  1400 :  0.23622552
Average loss on validation set at step  1500 :  0.08384403
Average loss on validation set at step  1600 :  0.096378

100%|██████████| 6000/6000 [02:44<00:00, 36.43it/s]


Average loss on validation set at step  4200 :  0.0022707982
Average loss on validation set at step  4300 :  0.0012483889
Average loss on validation set at step  4400 :  0.056115292
Average loss on validation set at step  4500 :  0.023486136
Average loss on validation set at step  4600 :  0.017043564
Average loss on validation set at step  4700 :  0.007946796
Average loss on validation set at step  4800 :  0.053300176
Average loss on validation set at step  4900 :  0.042417914
Average loss on validation set at step  5000 :  0.00390674
Average loss on validation set at step  5100 :  0.0033593713
Average loss on validation set at step  5200 :  0.008104741
Average loss on validation set at step  5300 :  0.02377075
Average loss on validation set at step  5400 :  0.06053863
Average loss on validation set at step  5500 :  0.031651556
Average loss on validation set at step  5600 :  0.023321664
Average loss on validation set at step  5700 :  0.077181116
Average loss on validation set at step  

100%|██████████| 6000/6000 [02:43<00:00, 36.70it/s]


Average loss on validation set at step  8300 :  0.022228017
Average loss on validation set at step  8400 :  0.0033733896
Average loss on validation set at step  8500 :  0.0096315
Average loss on validation set at step  8600 :  4.4895722e-07
Average loss on validation set at step  8700 :  0.24946086
Average loss on validation set at step  8800 :  0.00031287043
Average loss on validation set at step  8900 :  0.002291417
Average loss on validation set at step  9000 :  0.00044138537


TypeError: 'float' object cannot be interpreted as an integer